In [64]:
from langchain_community.llms.ollama import Ollama

model = Ollama(model='mistral-openorca')

In [65]:
import json

with open('../data/test.json', 'r') as raw_json:
    read = raw_json.read()
    parsed = json.loads(read)

old_method = parsed['src_method']
new_method = parsed['dst_method']
old_comment = parsed['src_javadoc']

In [87]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import PromptTemplate

response_schemas = [
    ResponseSchema(name="explanation", description="how the old comment should be changed"),
    ResponseSchema(name="newComment", description="the new comment"),
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


In [88]:
from langchain_core.output_parsers import StrOutputParser

prompt3 = PromptTemplate(
    template=
    '''
    Read the following Java method: ```{old_method}```
    Read the following javadoc comment belonging to the method mentioned before: {old_comment}
    Then the Java method was modified to ```{new_method}```
    Please tell me how should the old comment be changed to fit the new method.
    {format_instructions}''',
    input_variables=["old_method", "old_comment", "new_method"],
    partial_variables={"format_instructions": format_instructions},
)

chain3 = prompt3 | model | StrOutputParser()
r3 = chain3.invoke({
    'old_method': old_method,
    'new_method': new_method,
    'old_comment': old_comment
})

print(r3)

 {
  "explanation": "The original comment can be updated to better reflect the modified method. The main change is that the method now uses a builder for SSLConfig, which allows for more flexibility in creating the configuration object.",
  "newComment": "Configure an SSLConfig.Builder for the server using the legacy configuration."
}


In [75]:

from langchain_core.output_parsers import StrOutputParser

prompt1 = PromptTemplate(
    template='Please tell me the main difference(s) between the old method and the new method in one sentence. No more than 100 words. \nOld Method: ```{old_method}```\nNew Method: ```{new_method}```',
    input_variables=["old_method", "new_method"]
)
prompt2 = PromptTemplate(
    template=
    '''
    Read the following method modification: ```{modification}```
    Read the old javadoc comment: {old_comment}
    Tell me what the original comment is. 
    And then tell me how should the old comment be changed.
    * Do not compare
    * Do not mention the old method
    {format_instructions}''',
    input_variables=["modification", "old_comment"],
    partial_variables={"format_instructions": ''},
)

chain1 = prompt1 | model | StrOutputParser()
r1 = chain1.invoke({
    'new_method': new_method,
    'old_method': old_method,
})

print('Result1: ' + r1, end='\n\n')

chain2 = (
        prompt2
        | model
        | StrOutputParser()
)
result = chain2.invoke(
    {"modification": r1, "old_comment": old_comment}
)

print(f"Result2: {result}")

Result1:  The main difference is that the new method uses a builder pattern to configure SSL settings, providing a more modular and fluent approach for setting up SSL configurations compared to the previous method which used direct object mutation.
Result2:  The original comment was: "Configure a sslConfig for the server using the legacy configuration."
To change this to reflect the new approach with the builder pattern, it should now read: "Configure an SSL configuration for the server using a fluent and modular approach provided by the builder pattern."
